# Light-Weight CNN Tutorial (KOR)
## Origin: https://www.kaggle.com/alphasis/light-weight-cnn-lb-0-74
### Translated by siryuon

본 노트북의 목표 -> 경량 CNN 구축
본 노트북에서의 Input -> resampling된 .wav file(rate 8000)의 specgrams

본 캐글 노트북은 하드웨어 제한으로 인해 원래의 코드와는 다른 버전.

본 노트북으로 얻은 결과를 똑같이 얻으려면, epoch를 5로 설정, chop_audio(num=1000)을 설정하고, 모든 Conv Layer의 매개변수를 두 배 늘려야 함.

본 노트북은 Alex Ozerin의 baseline보다 약간 개선되었지만, 원본 .wav file(rate 16000)을 사용하면 더 높은 점수를 얻을 수 있을 것.

경량 CNN이기 때문에 성능이 제한됨.

성능을 높이기 위해 할 수 있는 방법들
  - resampling file 대신 원본 wav file 사용
  - chop_audio를 사용해 더 많은 'silence' wav file 만들기
  -  더 깊은 CNN을 사용하거나 RNN 사용
  -  더 긴 EPOCH으로 훈련

In [ ]:
import os
import numpy as np
from scipy.fftpack import fft
from scipy.io import wavfile
from scipy import signal
from glob import glob
import re
import pandas as pd
import gc
from scipy.io import wavfile

from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
import keras

Original Sample rate는 16000. 여기서는 8000으로 줄여준다.

In [ ]:
L = 16000
legal_labels = 'yes no up down left right on off stop go silence unknown'.split()

#src folders
root_path = r'..'
out_path = r'.'
model_path = r'.'
train_data_path = os.path.join(root_path, 'input', 'train', 'audio')
test_data_path = os.path.join(root_path, 'input', 'test', 'audio')

DavidS가 작성했던 function인 custom_fft, log_specgram을 불러온다.  
kaggle notebook: https://www.kaggle.com/davids1992/speech-representation-and-data-exploration

위 노트북의 1.1절(log_specgram), 1.5절(custom_fft)을 참고.

데이터 세트를 8000으로 리샘플링 가능 -> 중요하지 않은 정보를 버려 데이터 크기를 줄인다. -> fft 사용

In [ ]:
#FFT(Fast Fourier Transform)
def custom_fft(y, fs):
    T = 1.0 / fs
    N = y.shape[0]
    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    # FFT is simmetrical, so we take just the first half
    # FFT is also complex, to we take just the real part (abs)
    vals = 2.0/N * np.abs(yf[0:N//2])
    return xf, vals

#specgram? -> 스펙트로그램(spectrogram), 음성 데이터를 처리할 때 많이 볼 수 있다.
#파형과 스펙트럼의 특징이 결합된 것으로, x축은 시간, y축은 주파수, z축은 진폭을 나타냄. -> 소리의 스펙트럼을 시각화하여 그래프로 표현하는 기법

#specgram을 계산하는 함수(로그를 취한다)
#로그를 취함으로써 얻는 이점? -> plot을 훨씬 깔끔하게 할 수 있고, 사람들이 듣는 방식과 비슷하게 매칭이 될 것.
#로그에 대한 입력 값으로 0이 없는지 확인해야 한다.

def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

Train data folder 내의 모든 wav file을 가져오는 utility function

In [ ]:
def list_wavs_fname(dirpath, ext='wav'):
    print(dirpath)
    fpaths = glob(os.path.join(dirpath, r'*/*' + ext))
    pat = r'.+/(\w+)/\w+\.' + ext + '$'
    labels = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            labels.append(r.group(1))
    pat = r'.+/(\w+\.' + ext + ')$'
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            fnames.append(r.group(1))
    return labels, fnames

pad_audio function은 16000(1초) 미만의 오디오를 모두 동일한 길이로 만들기 위해 제로 패딩을 하는 함수.  

chop_audio function는 길이가 16000보다 큰 오디오(ex. 배경 소음 폴더의 wav file)을 16000으로 잘라준다. 또한, 매개변수 'num'이 지정된 하나의 큰 wav file에서 여러 chunk를 생성.

label_transform function은 레이블을 더미 값으로 변환. 레이블을 예측하기 위해 softmax와 함께 사용된다.

In [ ]:
def pad_audio(samples):
    if len(samples) >= L: return samples
    else: return np.pad(samples, pad_width=(L - len(samples), 0), mode='constant', constant_values=(0, 0))

def chop_audio(samples, L=16000, num=20):
    for i in range(num):
        beg = np.random.randint(0, len(samples) - L)
        yield samples[beg: beg + L]

def label_transform(labels):
    nlabels = []
    for label in labels:
        if label == '_background_noise_':
            nlabels.append('silence')
        elif label not in legal_labels:
            nlabels.append('unknown')
        else:
            nlabels.append(label)
    return pd.get_dummies(pd.Series(nlabels))

위에서 선언한 함수를 이용해 X_train과 y_train 생성.

label_index는 pandas가 더미 값을 생성하는 데 사용하는 index이므로 나중에 사용하기 위해 저장해야함.

In [ ]:
labels, fnames = list_wavs_fname(train_data_path)

new_sample_rate = 8000
y_train = []
x_train = []

for label, fname in zip(labels, fnames):
    sample_rate, samples = wavfile.read(os.path.join(train_data_path, label, fname))
    samples = pad_audio(samples)
    if len(samples) > 16000:
        n_samples = chop_audio(samples)
    else: n_samples = [samples]
    for samples in n_samples:
        resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        y_train.append(label)
        x_train.append(specgram)
x_train = np.array(x_train)
x_train = x_train.reshape(tuple(list(x_train.shape) + [1]))
y_train = label_transform(y_train)
label_index = y_train.columns.values
y_train = y_train.values
y_train = np.array(y_train)
del labels, fnames
gc.collect()

CNN 만들기.

생성된 specgram들은 (99, 81)의 shape인데, Conv2D 연산을 위해서는 모양을 변경해야 한다.

In [ ]:
input_shape = (99, 81, 1)
nclass = 12
inp = Input(shape=input_shape)
norm_inp = BatchNormalization()(inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(norm_inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Flatten()(img_1)

dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(img_1))
dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(dense_1))
dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

model = models.Model(inputs=inp, outputs=dense_1)
opt = optimizers.Adam()

model.compile(optimizer=opt, loss=losses.binary_crossentropy)
model.summary()

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=2017)
model.fit(x_train, y_train, batch_size=16, validation_data=(x_valid, y_valid), epochs=3, shuffle=True, verbose=2)

model.save(os.path.join(model_path, 'cnn.model'))

Test data가 너무 커서 RAM에 잘 돌아가지 않으므로 하나씩 처리해야 한다.

Generator인 test_data_generator는 CNN에 넣을 테스트 wav file의 배치를 생성.

In [ ]:
def test_data_generator(batch=16):
    fpaths = glob(os.path.join(test_data_path, '*wav'))
    i = 0
    for path in fpaths:
        if i == 0:
            imgs = []
            fnames = []
        i += 1
        rate, samples = wavfile.read(path)
        samples = pad_audio(samples)
        resampled = signal.resample(samples, int(new_sample_rate / rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        imgs.append(specgram)
        fnames.append(path.split('\\')[-1])
        if i == batch:
            i = 0
            imgs = np.array(imgs)
            imgs = imgs.reshape(tuple(list(imgs.shape) + [1]))
            yield fnames, imgs
    if i < batch:
        imgs = np.array(imgs)
        imgs = imgs.reshape(tuple(list(imgs.shape) + [1]))
        yield fnames, imgs
    raise StopIteration()

훈련된 모델을 사용해 예측 진행.

Kaggle은 테스트 데이터 제공 x -> 진행 종료.

이 노트북은 wav file의 Conv 연산에 대한 개괄적인 내용이라고 볼 수 있음.